In [1]:
import pickle
from urllib.request import urlopen
import certifi
import json
import pandas as pd
from datetime import datetime
import json
import queue
import threading

import fundamentalanalysis as fa
import fa_mods as famods
import fa_utils

with open('/home/craigc/.keys/financialmodelingprep.txt', 'r') as file:
    api_key = file.read()

In [3]:
all_companies = fa.available_companies(api_key)
tradeable_companies = famods.tradable_companies(api_key)

display(len(all_companies))
display(len(tradeable_companies))

70192

53942

In [3]:
exchanges = all_companies.groupby('exchangeShortName')['exchange'].agg('unique').reset_index().rename(columns={'exchangeShortName': 'short_name', 'unique': 'exchange'})

In [4]:
datestamp = datetime.now().strftime('%y%m%d')

with open(f'../data/all_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(all_companies, file)

with open(f'../data/all_tradable_symbols_{datestamp}.pkl', 'wb') as file:
    pickle.dump(tradeable_companies, file)

with open(f'../data/exchanges_{datestamp}.pkl', 'wb') as file:
    pickle.dump(exchanges, file)

In [5]:
lse_companies = all_companies[(all_companies.exchangeShortName == 'LSE')]
nyse_companies = all_companies[(all_companies.exchangeShortName == 'NYSE')]
nasdaq_companies = all_companies[(all_companies.exchangeShortName == 'NASDAQ')]

lse_stocks = lse_companies[(lse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nyse_stocks = nyse_companies[(nyse_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)
nasdaq_stocks = nasdaq_companies[(nasdaq_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

print(f"num LSE companies: {len(lse_companies)}")
print(f"num LSE stocks: {len(lse_stocks)}")
print()
print(f"num NYSE companies: {len(nyse_companies)}")
print(f"num NYSE stocks: {len(nyse_stocks)}")
print()
print(f"num NASDAQ companies: {len(nasdaq_companies)}")
print(f"num NASDAQ stocks: {len(nasdaq_stocks)}")

num LSE companies: 4829
num LSE stocks: 3001

num NYSE companies: 3614
num NYSE stocks: 3083

num NASDAQ companies: 6199
num NASDAQ stocks: 5473


In [6]:
thread_lock = threading.Lock()

def get_exchange_data(exchange: str):
    datestamp = datetime.now().strftime('%y%m%d')
    exchange_companies = all_companies[(all_companies.exchangeShortName == exchange)]
    exchange_stocks = exchange_companies[(exchange_companies.type == 'stock')].reset_index().drop(['exchange', 'exchangeShortName', 'type', 'price'], axis=1)

    symbols = exchange_stocks.symbol.tolist()
    stock_data = {}
    failed = []
    num_stocks = len(symbols)
    cnt = 0

    symbol_queue = queue.Queue()
    [symbol_queue.put(s) for s in symbols]
    symbol_queue.qsize()

    def worker():
        nonlocal cnt
        nonlocal stock_data
        nonlocal failed
        while True:
            with thread_lock:
                cnt += 1
                symbol = symbol_queue.get()        
                print(f'{exchange} [{cnt}/{num_stocks}]: getting {symbol} - f[{len(failed)}], s[{len(stock_data)}]')

            if symbol in stock_data:
                print(f"Already have {symbol}, skipping")
                continue

            try:
                data = fa_utils.get_ticker_data(symbol, api_key)
                stock_data[symbol] = data
                print(f"{symbol} SUCCESS")
            except:
                print(f"{symbol} FAILED")
                failed.append(symbol)

            symbol_queue.task_done()

    for x in range(4):
        name = "Thread_" + str(x)
        t = threading.Thread(name=name,target=worker, daemon=True).start()

    symbol_queue.join()

    print(len(failed))
    print(len(stock_data))

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'wb') as file:
        pickle.dump(stock_data, file)

    with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}_failed.pkl', 'wb') as file:
        pickle.dump(failed, file)

    return stock_data, failed

In [7]:
exchange = 'LSE'
data, failed = get_exchange_data(exchange)

LSE [1/3877]: getting MIRI.L - f[0], s[0]
LSE [2/3877]: getting STAN.L - f[0], s[0]
LSE [3/3877]: getting AIR3.L - f[0], s[0]
LSE [4/3877]: getting ESO.L - f[0], s[0]
AIR3.L SUCCESS
LSE [5/3877]: getting WISE.L - f[0], s[1]
MIRI.L SUCCESS
LSE [6/3877]: getting EMUM.L - f[0], s[2]
ESO.L SUCCESS
LSE [7/3877]: getting COMF.L - f[0], s[3]
STAN.L SUCCESS
LSE [8/3877]: getting 72HN.L - f[0], s[4]
WISE.L SUCCESS
LSE [9/3877]: getting TCAP.L - f[0], s[5]
EMUM.L SUCCESS
LSE [10/3877]: getting DMTR.L - f[0], s[6]
COMF.L SUCCESS
LSE [11/3877]: getting 0QL5.L - f[0], s[7]
72HN.L SUCCESS
LSE [12/3877]: getting WEIR.L - f[0], s[8]
DMTR.L SUCCESS
LSE [13/3877]: getting ASIA.L - f[0], s[9]
0QL5.L SUCCESS
LSE [14/3877]: getting DVT.L - f[0], s[10]
TCAP.L SUCCESS
LSE [15/3877]: getting 5QQE.L - f[0], s[11]
WEIR.L SUCCESS
LSE [16/3877]: getting BPCR.L - f[0], s[12]
5QQE.L SUCCESS
LSE [17/3877]: getting SFBE.L - f[0], s[13]
ASIA.L SUCCESS
LSE [18/3877]: getting KGF.L - f[0], s[14]
DVT.L SUCCESS
LSE [19/38

In [7]:
exchange = 'NASDAQ'
nasdaq_data, nasdaq_failed = get_exchange_data(exchange)

NYSE [1/3083]: getting KMI - f[0], s[0]
NYSE [2/3083]: getting GE - f[0], s[0]
NYSE [3/3083]: getting BAC - f[0], s[0]
NYSE [4/3083]: getting HPQ - f[0], s[0]
KMI SUCCESS
NYSE [5/3083]: getting CX - f[0], s[1]
BAC SUCCESS
NYSE [6/3083]: getting F - f[0], s[2]
HPQ SUCCESS
NYSE [7/3083]: getting SNAP - f[0], s[3]
GE SUCCESS
NYSE [8/3083]: getting WFC - f[0], s[4]
CX SUCCESS
NYSE [9/3083]: getting AIG - f[0], s[5]
SNAP SUCCESS
NYSE [10/3083]: getting T - f[0], s[6]
F SUCCESS
NYSE [11/3083]: getting C - f[0], s[7]
WFC SUCCESS
NYSE [12/3083]: getting VALE - f[0], s[8]
AIG SUCCESS
NYSE [13/3083]: getting MS - f[0], s[9]
T SUCCESS
NYSE [14/3083]: getting JPM - f[0], s[10]
VALE SUCCESS
NYSE [15/3083]: getting ORCL - f[0], s[11]
C SUCCESS
NYSE [16/3083]: getting NKE - f[0], s[12]
JPM SUCCESS
NYSE [17/3083]: getting PG - f[0], s[13]
MS SUCCESS
NYSE [18/3083]: getting BBD - f[0], s[14]
NKE SUCCESS
NYSE [19/3083]: getting PFE - f[0], s[15]
ORCL SUCCESS
NYSE [20/3083]: getting NOK - f[0], s[16]
BBD

KeyboardInterrupt: 

In [ ]:
exchange = 'NASDAQ'
nasdaq_data, nasdaq_failed = get_exchange_data(exchange)

In [ ]:
with open(f'../data/all_{exchange.lower()}_stock_data_{datestamp}.pkl', 'rb') as file:
    stock_data_2 = pickle.load(file)

print(len(stock_data_2))
display(stock_data_2)